In [6]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.metrics import make_scorer, roc_auc_score, log_loss
from sklearn.model_selection import GridSearchCV


df=pd.read_csv("MSME Data Modified_v1.9_featureEngineered.csv")

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# roc curve and auc score
from sklearn.datasets import make_classification

In [8]:
c = [29]
X = df.drop(df.columns[c], axis =1)
y = df['loan_default'].astype(int)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test   = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [11]:
## Apply RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
ytrain_pred = rf_model.predict_proba(X_train)
print('RF train roc-auc: {}'.format(roc_auc_score(y_train, ytrain_pred[:,1])))
ytest_pred = rf_model.predict_proba(X_test)
print('RF test roc-auc: {}'.format(roc_auc_score(y_test, ytest_pred[:,1])))

RF train roc-auc: 0.9999999999999999
RF test roc-auc: 0.6257303524969139


In [16]:
ytest_pred = rf_model.predict_proba(X_test)

In [17]:
ytest_pred

array([[0.92, 0.08],
       [0.85, 0.15],
       [0.76, 0.24],
       ...,
       [0.81, 0.19],
       [0.83, 0.17],
       [0.89, 0.11]])

In [19]:
# Sort on prediction (descending)
# Add row ids 
# Add decile 
ytest_pred = pd.DataFrame(ytest_pred[:,1],columns = {'predict'})
data= ytest_pred.sort_values(by='predict',ascending=False)
data['row_id'] = range(0,0+len(data))
data['decile'] = ( data['row_id'] / (len(data)/10) ).astype(int)
 # Check the count by decile
data.loc[data['decile'] == 10]=9
data['decile'].value_counts()
#create gains table
gains = data.groupby('decile')['target'].agg(['count','sum'])
gains.columns = ['count','actual']
gains
#add metrics to the gains table
gains['non_actual'] = gains['count'] - gains['actual']
gains['cum_count'] = gains['count'].cumsum()
gains['cum_actual'] = gains['actual'].cumsum()
gains['cum_non_actual'] = gains['non_actual'].cumsum()
gains['percent_cum_actual'] = (gains['cum_actual'] / np.max(gains['cum_actual'])).round(2)
gains['percent_cum_non_actual'] = (gains['cum_non_actual'] / np.max(gains['cum_non_actual'])).round(2)
gains['if_random'] = np.max(gains['cum_actual']) /10 
gains['if_random'] = gains['if_random'].cumsum()
gains['lift'] = (gains['cum_actual'] / gains['if_random']).round(2)
gains['K_S'] = np.abs( gains['percent_cum_actual'] - gains['percent_cum_non_actual'] ) * 100
gains['gain']=(gains['cum_actual']/gains['cum_count']*100).round(2)
gains = pd.DataFrame(gains)

KeyError: 'Column not found: target'